In [18]:
import pandas as pd
import numpy as np
data = pd.read_csv("/content/drive/MyDrive/PAYTM_historical_data.csv")
data

,Date,Open,High,Low,Close,Volume
0,2022-08-01T09:15:00+05:30,708.00,713.90,707.45,712.75,47041
1,2022-08-01T09:16:00+05:30,713.40,714.00,710.30,711.90,19707
2,2022-08-01T09:17:00+05:30,711.05,712.10,711.00,711.30,14889
3,2022-08-01T09:18:00+05:30,712.00,712.65,711.20,711.95,11883
4,2022-08-01T09:19:00+05:30,712.35,713.55,712.15,712.55,14532
...,...,...,...,...,...,...
184495,2023-07-31T15:25:00+05:30,644.80,644.80,644.15,644.15,19116
184496,2023-07-31T15:26:00+05:30,644.35,644.70,644.15,644.70,22111
184497,2023-07-31T15:27:00+05:30,644.70,644.70,644.35,644.40,24553
184498,2023-07-31T15:28:00+05:30,644.40,644.65,642.90,644.00,67930


In [22]:
import pandas as pd

data['Date'] = pd.to_datetime(data['Date'])

data['Actual_Returns'] = data['Close'].pct_change().fillna(0)

def basic_breakout_strategy(num_shares, data, breakout_buy_signal, breakout_sell_signal):
    # Extract relevant columns from the data (adjust column names if needed)
    close_prices = data['Close'].tolist()  # Assuming 'Close' column contains the prices

    # Calculate rolling high and low prices (adjust window size as needed)
    window_size = 20
    data['RollingHigh'] = data['High'].rolling(window=window_size).max()
    data['RollingLow'] = data['Low'].rolling(window=window_size).min()

    # Initialize variables
    position = None
    profit = 0

    # Iterate through data
    for i in range(len(data)):
        if data['Close'][i] > data['RollingHigh'][i]:
            if breakout_buy_signal == 1:
                position = "Buy"
                entry_price = close_prices[i]

        if data['Close'][i] < data['RollingLow'][i]:
            if breakout_sell_signal == 1 and position == "Buy":
                position = None
                exit_price = close_prices[i]
                profit += (exit_price - entry_price) * num_shares

    # Calculate profit percentage
    initial_investment = entry_price * num_shares
    profit_percentage = (profit / initial_investment) * 100

    return profit, profit_percentage


def trend_following_strategy(num_shares, data, breakout_buy_signal, breakout_sell_signal):
    # Extract relevant columns from the data (adjust column names if needed)
    close_prices = data['Close'].tolist()  # Assuming 'Close' column contains the prices

    # Calculate rolling average (adjust window size as needed)
    window_size = 50
    data['RollingAverage'] = data['Close'].rolling(window=window_size).mean()

    # Initialize variables
    position = None
    profit = 0

    # Iterate through data
    for i in range(len(data)):
        if data['Close'][i] > data['RollingAverage'][i]:
            if breakout_buy_signal == 1:
                position = "Buy"
                entry_price = close_prices[i]

        if data['Close'][i] < data['RollingAverage'][i]:
            if breakout_sell_signal == 1 and position == "Buy":
                position = None
                exit_price = close_prices[i]
                profit += (exit_price - entry_price) * num_shares

    # Calculate profit percentage
    initial_investment = entry_price * num_shares
    profit_percentage = (profit / initial_investment) * 100

    return profit, profit_percentage

def momentum_trading_strategy(num_shares, data, breakout_buy_signal, breakout_sell_signal):
    # Extract relevant columns from the data (adjust column names if needed)
    close_prices = data['Close'].tolist()  # Assuming 'Close' column contains the prices

    # Calculate momentum (adjust window size as needed)
    window_size = 10
    data['Momentum'] = data['Close'].diff(window_size)

    # Initialize variables
    position = None
    profit = 0

    # Iterate through data
    for i in range(window_size, len(data)):
        if data['Momentum'][i] > 0 and data['Momentum'][i - 1] <= 0:
            if breakout_buy_signal == 1:
                position = "Buy"
                entry_price = close_prices[i]

        if data['Momentum'][i] < 0 and data['Momentum'][i - 1] >= 0:
            if breakout_sell_signal == 1 and position == "Buy":
                position = None
                exit_price = close_prices[i]
                profit += (exit_price - entry_price) * num_shares

    # Calculate profit percentage
    initial_investment = entry_price * num_shares
    profit_percentage = (profit / initial_investment) * 100

    return profit, profit_percentage

def pivot_point_trading_strategy(num_shares, data, breakout_buy_signal, breakout_sell_signal):
    # Extract relevant columns from the data (adjust column names if needed)
    high_prices = data['High'].tolist()
    low_prices = data['Low'].tolist()
    close_prices = data['Close'].tolist()

    # Calculate pivot points and support/resistance levels
    pivot_point = (high_prices[0] + low_prices[0] + close_prices[0]) / 3
    first_support = (pivot_point * 2) - high_prices[0]
    first_resistance = (pivot_point * 2) - low_prices[0]

    # Initialize variables
    position = None
    profit = 0

    # Iterate through data
    for i in range(1, len(data)):
        if close_prices[i - 1] < pivot_point and close_prices[i] > pivot_point:
            if breakout_buy_signal == 1:
                position = "Buy"
                entry_price = close_prices[i]

        if close_prices[i - 1] > pivot_point and close_prices[i] < pivot_point:
            if breakout_sell_signal == 1 and position == "Buy":
                position = None
                exit_price = close_prices[i]
                profit += (exit_price - entry_price) * num_shares

    # Calculate profit percentage
    initial_investment = entry_price * num_shares
    profit_percentage = (profit / initial_investment) * 100

    return profit, profit_percentage

def range_breakout_strategy(num_shares, data, breakout_buy_signal, breakout_sell_signal):
    # Extract relevant columns from the data (adjust column names if needed)
    high_prices = data['High'].tolist()
    low_prices = data['Low'].tolist()
    close_prices = data['Close'].tolist()

    # Calculate the average range (adjust window size as needed)
    window_size = 20
    data['AverageRange'] = data['High'] - data['Low']
    average_range = data['AverageRange'].rolling(window=window_size).mean()

    # Initialize variables
    position = None
    profit = 0

    # Iterate through data
    for i in range(window_size, len(data)):
        if close_prices[i] > high_prices[i - 1] and close_prices[i] > average_range[i - 1]:
            if breakout_buy_signal == 1:
                position = "Buy"
                entry_price = close_prices[i]

        if close_prices[i] < low_prices[i - 1] and close_prices[i] < average_range[i - 1]:
            if breakout_sell_signal == 1 and position == "Buy":
                position = None
                exit_price = close_prices[i]
                profit += (exit_price - entry_price) * num_shares

    # Calculate profit percentage
    initial_investment = entry_price * num_shares
    profit_percentage = (profit / initial_investment) * 100

    return profit, profit_percentage

def options_straddle_strategy(num_shares, data, breakout_buy_signal, breakout_sell_signal):
    # Extract relevant columns from the data (adjust column names if needed)
    close_prices = data['Close'].tolist()  # Assuming 'Close' column contains the prices

    # Initialize variables
    position = None
    profit = 0

    # Iterate through data
    for i in range(1, len(data)):
        if breakout_buy_signal == 1 and breakout_sell_signal == 1:
            # Buy a call option and a put option
            if position is None:
                position = "Buy"
                entry_price = close_prices[i]

            if position == "Buy":
                exit_price = close_prices[i]
                profit += (exit_price - entry_price) * num_shares

    # Calculate profit percentage
    initial_investment = entry_price * num_shares
    profit_percentage = (profit / initial_investment) * 100

    return profit, profit_percentage

def gap_trading_strategy(num_shares, data, breakout_buy_signal, breakout_sell_signal):
    # Extract relevant columns from the data (adjust column names if needed)
    open_prices = data['Open'].tolist()  # Assuming 'Open' column contains the opening prices
    close_prices = data['Close'].tolist()  # Assuming 'Close' column contains the closing prices

    # Initialize variables
    position = None
    profit = 0

    # Iterate through data (starting from the second data point)
    for i in range(1, len(data)):
        if open_prices[i] > close_prices[i - 1]:
            # Gap up
            if breakout_buy_signal == 1:
                position = "Buy"
                entry_price = open_prices[i]

        if open_prices[i] < close_prices[i - 1]:
            # Gap down
            if breakout_sell_signal == 1 and position == "Buy":
                position = None
                exit_price = open_prices[i]
                profit += (exit_price - entry_price) * num_shares

    # Calculate profit percentage
    initial_investment = entry_price * num_shares
    profit_percentage = (profit / initial_investment) * 100

    return profit, profit_percentage

def seasonal_trading_strategy(num_shares, data, breakout_buy_signal, breakout_sell_signal):
    # Extract relevant columns from the data (adjust column names if needed)
    close_prices = data['Close'].tolist()  # Assuming 'Close' column contains the prices

    # Initialize variables
    position = None
    profit = 0

    # Iterate through data
    for i in range(1, len(data)):
        # Define the seasonal pattern (e.g., buying in certain months)
        seasonal_pattern = [0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]  # Buy in months 3 and 4

        if breakout_buy_signal == 1 and breakout_sell_signal == 1:
            if seasonal_pattern[i % 12] == 1:
                # Buy according to the seasonal pattern
                position = "Buy"
                entry_price = close_prices[i]

        if position == "Buy":
            exit_price = close_prices[i]
            profit += (exit_price - entry_price) * num_shares

            # Reset position after selling
            position = None

    # Calculate profit percentage
    initial_investment = entry_price * num_shares
    profit_percentage = (profit / initial_investment) * 100

    return profit, profit_percentage

def volatility_breakout_strategy(num_shares, data, breakout_buy_signal, breakout_sell_signal):
    # Extract relevant columns from the data (adjust column names if needed)
    high_prices = data['High'].tolist()
    low_prices = data['Low'].tolist()
    close_prices = data['Close'].tolist()

    # Calculate average true range (ATR) as a measure of volatility (adjust window size as needed)
    window_size = 14
    true_range = [max(high_prices[i] - low_prices[i], abs(high_prices[i] - close_prices[i - 1]), abs(low_prices[i] - close_prices[i - 1])) for i in range(1, len(data))]
    atr = [sum(true_range[i - window_size + 1 : i + 1]) / window_size for i in range(window_size - 1, len(true_range))]

    # Initialize variables
    position = None
    profit = 0

    # Iterate through data (starting from the window size)
    for i in range(window_size, len(data)):
        upper_threshold = close_prices[i - 1] + atr[i - window_size] * 1.5
        lower_threshold = close_prices[i - 1] - atr[i - window_size] * 1.5

        if close_prices[i] > upper_threshold:
            # Buy if price breaks above the upper threshold
            if breakout_buy_signal == 1:
                position = "Buy"
                entry_price = close_prices[i]

        if close_prices[i] < lower_threshold:
            # Sell if price breaks below the lower threshold
            if breakout_sell_signal == 1 and position == "Buy":
                position = None
                exit_price = close_prices[i]
                profit += (exit_price - entry_price) * num_shares

    # Calculate profit percentage
    initial_investment = entry_price * num_shares
    profit_percentage = (profit / initial_investment) * 100

    return profit, profit_percentage

def trend_reversal_strategy(num_shares, data, breakout_buy_signal, breakout_sell_signal):
    # Extract relevant columns from the data (adjust column names if needed)
    close_prices = data['Close'].tolist()  # Assuming 'Close' column contains the prices

    # Calculate moving averages (adjust window sizes as needed)
    short_window = 20
    long_window = 50
    data['Short_MA'] = data['Close'].rolling(window=short_window).mean()
    data['Long_MA'] = data['Close'].rolling(window=long_window).mean()

    # Initialize variables
    position = None
    profit = 0

    # Iterate through data
    for i in range(max(short_window, long_window), len(data)):
        if data['Short_MA'][i] > data['Long_MA'][i] and data['Short_MA'][i - 1] <= data['Long_MA'][i - 1]:
            if breakout_buy_signal == 1:
                position = "Buy"
                entry_price = close_prices[i]

        if data['Short_MA'][i] < data['Long_MA'][i] and data['Short_MA'][i - 1] >= data['Long_MA'][i - 1]:
            if breakout_sell_signal == 1 and position == "Buy":
                position = None
                exit_price = close_prices[i]
                profit += (exit_price - entry_price) * num_shares

    # Calculate profit percentage
    initial_investment = entry_price * num_shares
    profit_percentage = (profit / initial_investment) * 100

    return profit, profit_percentage


# Get user inputs
strategy_name = input("Enter the name of the strategy\n(basic breakout trading,\ntrend following strategy,\nmomentum trading strategy,\npivot point trading strategy,\nrange breakout strategy,\noptions straddle strategy,\ngap trading strategy,\nseasonal trading strategy,\nvolatility breakout strategy,\ntrend reversal strategy): ")
num_shares = int(input("Enter the number of shares: "))
breakout_buy_signal = int(input("Enter the value(0 or 1) for Breakout_Buy_Signal: "))
breakout_sell_signal = int(input("Enter the value(0 or 1) for Breakout_Sell_Signal: "))

# Check if the selected strategy is in the dictionary
strategy_functions = {
    "basic breakout trading": basic_breakout_strategy,
    "trend following strategy": trend_following_strategy,
    "momentum trading strategy":momentum_trading_strategy,
    "pivot point trading strategy":pivot_point_trading_strategy,
    "range breakout strategy":range_breakout_strategy,
    "options straddle strategy":options_straddle_strategy,
    "gap trading strategy":gap_trading_strategy,
    "seasonal trading strategy":seasonal_trading_strategy,
    "volatility breakout strategy":volatility_breakout_strategy,
    "trend reversal strategy":trend_reversal_strategy,
}

if strategy_name.lower() in strategy_functions:
    calculate_profit_function = strategy_functions[strategy_name.lower()]
    profit, profit_percentage = calculate_profit_function(num_shares, data, breakout_buy_signal, breakout_sell_signal)

    # Print results
    print(f"Profit: {profit:.2f}")
    print(f"Profit Percentage: {profit_percentage:.2f}%")
else:
    print(f"Strategy '{strategy_name}' not found.")


Enter the name of the strategy
(basic breakout trading,
trend following strategy,
momentum trading strategy,
pivot point trading strategy,
range breakout strategy,
options straddle strategy,
gap trading strategy,
seasonal trading strategy,
volatility breakout strategy,
trend reversal strategy): trend reversal strategy
Enter the number of shares: 3
Enter the value(0 or 1) for Breakout_Buy_Signal: 1
Enter the value(0 or 1) for Breakout_Sell_Signal: 0
Profit: 0.00
Profit Percentage: 0.00%
